In [56]:
from dotenv import load_dotenv
load_dotenv()

# Use the variable with:
import os
API_KEY = os.getenv("API_KEY")
STEAM_ID = os.getenv("STEAM_ID")

import requests
import pandas as pd
from tqdm.auto import tqdm

General flow:
- Start with my Steam ID.
- Get a list of my games and save it in a dataframe.
- Everytime the dataframe is updated, save it in a csv file.
- Append the SteamID to the list of already visited SteamIDs.
- For each SteamID, get a list of their friends, check if they are already in the list of visited SteamIDs, and if not, add them to the list of SteamIDs to visit.
- Save the list of SteamIDs to visit in a csv file.
- Repeat the process.

In [47]:
# Takes a Steam ID and returns a DataFrame row with all of the user's games and playtime
def get_games(steam_id):
    url = f'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={API_KEY}&steamid={steam_id}&format=json'
    response = requests.get(url)
    data = response.json()
    data = data['response']
    # only extract app_id and playtime_forever from data['games']
    clean_data = []
    if 'games' not in data:
        return None
    for game in data['games']:
        clean_data.append({'app_id': game['appid'], 'playtime_forever': game['playtime_forever']})
    games = pd.DataFrame({'Total Games': data['game_count'], 'Games': [clean_data]}, index=[steam_id])
    return games

In [39]:
# Takes a Steam ID and returns a list of the user's friends
def get_friends(steam_id, visited_friends):
    url = f'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={API_KEY}&steamid={steam_id}&relationship=friend'
    response = requests.get(url)
    data = response.json()
    if 'friendslist' not in data:
        return None
    friends_to_visit = []
    for friend in data['friendslist']['friends']:
        if friend['steamid'] not in visited_friends:
            friends_to_visit.append(friend['steamid'])
    return friends_to_visit

In [40]:
# Saves all already visited friend IDs and saves it to a file
def save_visited_friends(visited_friends):
    with open('visited_friends.txt', 'w') as f:
        for friend in visited_friends:
            f.write(f'{friend}\n')

# Saves the queue of friends to visit to a file
def save_friends_queue(friends_queue):
    with open('friends_queue.txt', 'w') as f:
        for friend in friends_queue:
            f.write(f'{friend}\n')

# Appends the games dataframe to a csv file (or creates a new one if it doesn't exist)
def save_games(games, filename, overwrite=False):
    if not os.path.isfile(filename) or overwrite:
        games.to_csv(filename, header=['Total Games', 'Games'], index_label='Steam ID')
    else:
        games.to_csv(filename, mode='a', header=False)

In [73]:
overwrite = False # Set to True to start from scratch

# Read queue.txt if exists, otherwise start with STEAM_ID
if os.path.exists('friends_queue.txt') and not overwrite:
    with open('friends_queue.txt', 'r') as f:
        friends_queue = f.read().splitlines()
else:
    friends_queue = [STEAM_ID]

# Read visited_friends.txt if exists
if os.path.exists('visited_friends.txt') and not overwrite:
    with open('visited_friends.txt', 'r') as f:
        visited_friends = f.read().splitlines()
else:
    visited_friends = []

if overwrite:
    if os.path.exists('user_games_data.csv'):
        os.remove('user_games_data.csv')

iterations = 1500

for i in tqdm(range(iterations)):
    user = friends_queue.pop(0)
    visited_friends.append(user)

    print(f"Queue: {len(friends_queue)} | Visited total: {len(visited_friends)}")

    # Get games and append them to the csv
    games = get_games(user)

    
    # Get (not yet visited) friends of the user and append them to the queue
    friends = get_friends(user, visited_friends)
    if friends is not None and len(friends) + len(friends_queue) < 3000:
        friends_queue.extend(friends)

    if games is not None:
        save_games(games, 'user_games_data.csv')

    # Save the visited friends and the friends queue
    save_visited_friends(visited_friends)
    save_friends_queue(friends_queue)
    

  0%|          | 0/1500 [00:00<?, ?it/s]

Queue: 2966 | Visited total: 5001
Queue: 2965 | Visited total: 5002
Queue: 2964 | Visited total: 5003
Queue: 2968 | Visited total: 5004
Queue: 2974 | Visited total: 5005
Queue: 2973 | Visited total: 5006
Queue: 2972 | Visited total: 5007
Queue: 2971 | Visited total: 5008
Queue: 2970 | Visited total: 5009
Queue: 2969 | Visited total: 5010
Queue: 2968 | Visited total: 5011
Queue: 2967 | Visited total: 5012
Queue: 2966 | Visited total: 5013
Queue: 2965 | Visited total: 5014
Queue: 2964 | Visited total: 5015
Queue: 2963 | Visited total: 5016
Queue: 2962 | Visited total: 5017
Queue: 2961 | Visited total: 5018
Queue: 2960 | Visited total: 5019
Queue: 2959 | Visited total: 5020
Queue: 2958 | Visited total: 5021
Queue: 2957 | Visited total: 5022
Queue: 2956 | Visited total: 5023
Queue: 2955 | Visited total: 5024
Queue: 2954 | Visited total: 5025
Queue: 2953 | Visited total: 5026
Queue: 2966 | Visited total: 5027
Queue: 2984 | Visited total: 5028
Queue: 2983 | Visited total: 5029
Queue: 2983 | 

In [74]:
games = pd.read_csv('user_games_data.csv', index_col=0)

In [77]:
games.loc[games['Total Games'] < 50]

,Total Games,Games
Steam ID,,
76561197978985924,21,"[{'app_id': 10, 'playtime_forever': 20007}, {'..."
76561198014077583,40,"[{'app_id': 620, 'playtime_forever': 108}, {'a..."
76561198057605973,21,"[{'app_id': 40800, 'playtime_forever': 4469}, ..."
76561198081514151,5,"[{'app_id': 6060, 'playtime_forever': 155}, {'..."
76561198100576827,7,"[{'app_id': 12210, 'playtime_forever': 1171}, ..."
...,...,...
76561198080201309,26,"[{'app_id': 12120, 'playtime_forever': 685}, {..."
76561198180651865,33,"[{'app_id': 2990, 'playtime_forever': 1858}, {..."
76561198419185267,29,"[{'app_id': 105450, 'playtime_forever': 3521},..."


In [75]:
games

,Total Games,Games
Steam ID,,
76561198280830500,1152,"[{'app_id': 4000, 'playtime_forever': 77748}, ..."
76561197962287503,5610,"[{'app_id': 10, 'playtime_forever': 3}, {'app_..."
76561197977849691,11911,"[{'app_id': 10, 'playtime_forever': 1}, {'app_..."
76561197978985924,21,"[{'app_id': 10, 'playtime_forever': 20007}, {'..."
76561197982447568,949,"[{'app_id': 20, 'playtime_forever': 330}, {'ap..."
...,...,...
76561198402515194,91,"[{'app_id': 4000, 'playtime_forever': 188}, {'..."
76561198419185267,29,"[{'app_id': 105450, 'playtime_forever': 3521},..."
76561198803191321,34,"[{'app_id': 242760, 'playtime_forever': 557}, ..."


GetOwnedGames - include_played_free_games
https://developer.valvesoftware.com/wiki/Steam_Web_API#GetPlayerSummaries_.28v0001.29

data przeczyścić bo zajmuje w cholerę miejsca